In [1]:
from tracking import PeopleTracker
import csv
import time
import matplotlib.pyplot as plt
import math
import os

track = PeopleTracker(debug = False)

class Detection:
    def __init__(self, x: float, y: float, orientation: float):
        self.x = x
        self.y = y
        self.orientation = orientation


In [2]:
with open('./SanityCheck.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    ID = 0
    detections = []
    timestamp = 0
    predictNum = 0
    for row in csv_reader:
        print(f"Row ID: {row[0]}, x: {row[2]}, y: {row[3]}, theta: {row[4]} - length of detections is {len(detections)}")
        if line_count == 0:
            #print(f'Column names are {", ".join(row)}')
            line_count += 1
        
        elif int(row[0]) == ID or len(detections) == 0:
            detections.append(Detection(float(row[2]), float(row[3]), float(row[4])))
            timestamp = float(row[1])
            line_count += 1
            ID = int(row[0])

        else:
            if predictNum:
                track.predict()
                for person in track.personList:
                    plt.arrow(float(person.personX), float(person.personY), math.cos(float(person.personTheta)), math.sin(float(person.personTheta)),
                        head_width = 0.02,
                        width = 0.01,
                        ec ='red')

            if len(detections):
                track.update(detections, timestamp)
                
                for detection in detections:
                    plt.arrow(float(detection.x), float(detection.y), math.cos(float(detection.orientation)), math.sin(float(detection.orientation)),
                        head_width = 0.02,
                        width = 0.01,
                        ec ='blue')
                    
                detections = []
                for person in track.personList:
                    plt.arrow(float(person.personX), float(person.personY), math.cos(float(person.personTheta)), math.sin(float(person.personTheta)),
                        head_width = 0.02,
                        width = 0.01,
                        ec ='green')
                #plt.show()
                plt.savefig(f"./images/{ID}.jpg")
                plt.clf()
                # time.sleep(1)

            detections.append(Detection(float(row[2]), float(row[3]), float(row[4])))
            timestamp = float(row[1])
            line_count += 1
            predictNum += 1
            ID = int(row[0])

    
    print(f'Processed {line_count} lines.')

    

Row ID: ImageID, x: PersonX, y: PersonY, theta: Orientation - length of detections is 0
Row ID: 0, x: 2.066, y: -0.232, theta: 2.048 - length of detections is 0
Row ID: 1, x: 2.047, y: -0.239, theta: 2.118 - length of detections is 1
Row ID: 2, x: 2.045, y: -0.255, theta: 2.074 - length of detections is 1
Row ID: 3, x: 2.147, y: -0.307, theta: 1.889 - length of detections is 1
Row ID: 4, x: 2.136, y: -0.2, theta: 2.849 - length of detections is 1
Row ID: 5, x: 2.18, y: -0.124, theta: -2.819 - length of detections is 1
Row ID: 6, x: 2.183, y: -0.126, theta: -2.93 - length of detections is 1
Row ID: 7, x: 2.193, y: -0.131, theta: -2.889 - length of detections is 1
Row ID: 8, x: 2.161, y: -0.223, theta: 2.948 - length of detections is 1
Row ID: 9, x: 2.141, y: -0.261, theta: 2.793 - length of detections is 1
Row ID: 10, x: 2.15, y: -0.279, theta: 2.497 - length of detections is 1
Row ID: 11, x: 2.121, y: -0.311, theta: 2.422 - length of detections is 1
Row ID: 12, x: 2.048, y: -0.352, the

<Figure size 640x480 with 0 Axes>

In [3]:
import cv2
import numpy as np
import os

# assign directory
directory = 'images'

width = 640
height = 480

# choose codec according to format needed
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('video.avi', fourcc, 2, (width, height))

files = []
# iterate over files in directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        files.append(f)

files.sort()

print(files)

for file in range(0, 170):
    img = cv2.imread(f"./images/{file}.jpg")
    video.write(img)
cv2.destroyAllWindows()
video.release()

['images\\0.jpg', 'images\\1.jpg', 'images\\10.jpg', 'images\\100.jpg', 'images\\101.jpg', 'images\\102.jpg', 'images\\103.jpg', 'images\\104.jpg', 'images\\105.jpg', 'images\\106.jpg', 'images\\107.jpg', 'images\\108.jpg', 'images\\109.jpg', 'images\\11.jpg', 'images\\110.jpg', 'images\\111.jpg', 'images\\112.jpg', 'images\\113.jpg', 'images\\114.jpg', 'images\\115.jpg', 'images\\116.jpg', 'images\\117.jpg', 'images\\118.jpg', 'images\\119.jpg', 'images\\12.jpg', 'images\\120.jpg', 'images\\121.jpg', 'images\\122.jpg', 'images\\123.jpg', 'images\\124.jpg', 'images\\125.jpg', 'images\\126.jpg', 'images\\127.jpg', 'images\\128.jpg', 'images\\129.jpg', 'images\\13.jpg', 'images\\130.jpg', 'images\\131.jpg', 'images\\132.jpg', 'images\\133.jpg', 'images\\134.jpg', 'images\\135.jpg', 'images\\136.jpg', 'images\\137.jpg', 'images\\138.jpg', 'images\\139.jpg', 'images\\14.jpg', 'images\\140.jpg', 'images\\141.jpg', 'images\\142.jpg', 'images\\143.jpg', 'images\\144.jpg', 'images\\145.jpg', '